In [7]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [8]:
!python -V

Python 3.10.13


In [15]:
import pickle
import pandas as pd

In [16]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [17]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [21]:
df['predicted_duration']=y_pred

In [23]:
#Standard deviation 
df['predicted_duration'].std()

6.2474897941993985

In [26]:
year=2023
month=3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [29]:
df_result=pd.DataFrame()

In [30]:
df_result['ride_id']=df['ride_id']
df_result['predicted_duration']=df['predicted_duration']

In [31]:
df_result

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [34]:
!mkdir output
#Create a file with name 'output_file' in output directory.



In [38]:
output_file= 'output/output_file'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [39]:
import os
file_stats = os.stat(output_file)

In [40]:
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

File Size in MegaBytes is 65.46199798583984
